Hyperparameter tuning is the process of optimizing the "settings" of your neural network that aren’t learned during training. These include factors like learning rate, number of layers, neurons per layer, dropout rate, batch size, optimizer type, and so on. The goal is to find the configuration that gives the best performance on your validation set

# Key Steps for Hyperparameter Tuning


1.   Define Your Search Space:
2.   Choose a Search Strategy:


*   Grid Search: Exhaustively try every combination in the defined space (can be computationally expensive).
*   
Random Search: Randomly sample hyperparameter combinations; often more efficient.


* Bayesian Optimization/Hyperband: More advanced methods that can intelligently explore the space based on past evaluations.

3. Automate the Process:
Use libraries or frameworks that facilitate hyperparameter tuning, such as:

* Keras Tuner for TensorFlow/Keras models.
* Hyperopt or Optuna for broader use.
* Ray Tune for distributed hyperparameter tuning.

4. Evaluate and Iterate:
* Run your tuning process, review the best-performing configurations, and then consider further fine-tuning within the promising hyperparameter ranges.

In [1]:
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import kerastuner as kt

def build_model(hp):
    model = keras.Sequential()

    # Tune the number of layers.
    for i in range(hp.Int('num_layers', 2, 4)):
        # Tune the number of units in each layer.
        model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=64, max_value=256, step=64),
                               activation='relu'))
        # Optionally add dropout.
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(layers.Dense(10, activation='softmax'))  # For example, 10 classes.

    # Tune the learning rate for the optimizer.
    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Instantiate the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='hyperparam_tuning_example'
)

# Prepare your data
(x_train, y_train), (x_val, y_val) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28) / 255.0
x_val = x_val.reshape(-1, 28 * 28) / 255.0

# Run the hyperparameter search
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

# Get the best model hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best learning rate: {best_hps.get('learning_rate')}")
print(f"Best number of layers: {best_hps.get('num_layers')}")


Trial 10 Complete [00h 00m 35s]
val_accuracy: 0.9333000183105469

Best val_accuracy So Far: 0.9778000116348267
Total elapsed time: 00h 06m 23s
Best learning rate: 0.001
Best number of layers: 2
